In [2]:
#Params 
param_stationID = '260'
param_from_date = '2006'
param_to_date = '2025' 
param_vars = "average_windspeed, average_temperature, minimum_temperature, maximum_temperature, total_precipitation,
                average_winddirection, relative_humidity, minimum_humidity, maximum_humidity"

param_stationID_spinup = '260'
param_from_date_spinup = '1995'
param_to_date_spinup = '2005' 
param_vars_spinup = "average_windspeed, average_temperature, minimum_temperature, maximum_temperature, total_precipitation,
                    average_winddirection, relative_humidity, minimum_humidity, maximum_humidity"

In [7]:
#KNMI data retriever
# ---
# NaaVRE:
#  cell:
#   params:
#    - param_stationID:
#       type: String
#       default_value: '260'
#    - param_from_date:
#       type: String
#       default_value: '2006'
#    - param_to_date:
#       type: String
#       default_value: '2025'
#    - param_vars: 
#       type: String
#       default_value: 'average_windspeed, average_temperature, minimum_temperature, maximum_temperature, total_precipitation, average_winddirection, relative_humidity, minimum_humidity, maximum_humidity'
#    - param_stationID_spinup:
#       type: String
#       default_value: '260'
#    - param_from_date_spinup:
#       type: String
#       default_value: '1995'
#    - param_to_date_spinup:
#       type: String
#       default_value: '2005'
#    - param_vars_spinup: 
#       type: String
#       default_value: 'average_windspeed, average_temperature, minimum_temperature, maximum_temperature, total_precipitation, average_winddirection, relative_humidity, minimum_humidity, maximum_humidity'
#   outputs:
#    - KNMI_daily_datafile: String
#    - KNMI_spinup_datafile: String
# ...

#------------------
##Install packages
install.packages("readr")
install.packages("dplyr")
install.packages("tidyr")
install.packages("stringr")
install.packages("lubridate")

#Load packages
library(readr)
library(dplyr)
library(tidyr)
library(stringr)
library(lubridate)

#Create directory
dir.create("/tmp/data")
#-------------------

#Parameter validation - check whether all parameters were filled in by user
if (param_stationID == "" ||
    param_from_date == "" ||
    param_to_date == "" ||
    length(param_vars) == 0) {
  stop("All parameters must be provided by the user.")
}

#Split climate variables to make columns and rename so that function can use set parameter values 
param_vars <- unlist(strsplit(param_vars, ",\\s*"))
stationID  <- param_stationID
from       <- param_from_date
to         <- param_to_date

param_vars[param_vars == "average_windspeed"]     <- "FG"
param_vars[param_vars == "average_temperature"]   <- "TG"
param_vars[param_vars == "minimum_temperature"]   <- "TN"
param_vars[param_vars == "maximum_temperature"]   <- "TX"
param_vars[param_vars == "total_precipitation"]   <- "RH"
param_vars[param_vars == "average_winddirection"] <- "DDVEC"
param_vars[param_vars == "relative_humidity"]      <- "UG"
param_vars[param_vars == "minimum_humidity"]      <- "UN"
param_vars[param_vars == "maximum_humidity"]      <- "UX"

#Function to get the daily KNMI data, includes checks whether the dates are filled in correctly and logically 
get_data <- function(stationID, from, to, vars) {
  
  #Date parsing - checking whether logical 
  if (!is.character(from) ||
      !is.character(to) ||
      stringr::str_length(from) %% 2 == 1 ||
      stringr::str_length(to) %% 2 == 1) {
    stop(
      "The values for 'from' and 'to' must be strings in the format 'YYYY', 'YYYYMM' or 'YYYYMMDD'."
    )
  }
  
  if (stringr::str_length(from) == 6) {
    from_date <- paste0(from, "01")
  } else if (stringr::str_length(from) == 4) {
    from_date <- paste0(from, "0101")
  } else {
    from_date <- from
  }
  
  if (stringr::str_length(to) == 8) {
    to_date <- to
  } else {
    if (stringr::str_length(to) == 6) {
      to <- paste0(to, "01")
    } else if (stringr::str_length(to) == 4) {
      to <- paste0(to, "1231")
    }
    
    to_date <- lubridate::ymd(to) %>%
      lubridate::ceiling_date(unit = "month") - 1
    
    to_date <- gsub("-", "", as.character(to_date))
  }
  
  if (as.numeric(to_date) < as.numeric(from_date)) {
    stop("'from' must be earlier than or equal to 'to'")
  }
  
  # Station validation - check whether station can be selected. The list contains all the KNMI weather stations that collect daily data 
  selectable_stations <- c("260", "275", "209", "210", "215", "225", "229", "235", "240", "242", "248", "249", "251", "257", "258", "260", "265", 
                           "267", "269", "270", "273", "275", "277", "278", "279", "280", "283", "285", "286", "290", "308", "310", "311", "312", 
                           "313", "315", "316", "319", "323", "324", "330", "331", "340", "343", "344", "348", "350", "356", "370", "375", "377",
                           "380", "391", "392")  
  stationID <- as.character(stationID)
  
  if (!stationID %in% selectable_stations) {
    stop(
      paste0(
        "Invalid stationID. Choose one of: ",
        paste(selectable_stations, collapse = ", ")
      )
    )
  }
  
  #Build URL to retrieve the KNMI data from the website 
  baseURL <- "https://www.daggegevens.knmi.nl/klimatologie/daggegevens"
  URL <- paste0(
    baseURL,
    "?start=", from_date,
    "&end=", to_date,
    "&stns=", stationID,
    "&ALL"
  )
  
  #Download data from KNMI 
  data_daily <- readr::read_csv(
    URL,
    col_names = FALSE,
    comment = "#",
    show_col_types = FALSE
  ) %>%
    dplyr::as_tibble()
  
  colnames(data_daily) <- c(
    "STN", "YYYYMMDD", "FG", "TG", "TN",
    "TX", "RH", "DDVEC", "UG", "UX", "UN"
  )
  
  # Select only requested variables as indicated when parameters were set 
  data_daily <- data_daily %>%
    tidyr::drop_na(dplyr::all_of(vars)) %>%
    select(dplyr::all_of(c("YYYYMMDD", vars)))
  
  # Scaling because KNMI works in 0.1
  if ("FG" %in% vars) data_daily$FG <- data_daily$FG / 10
  if ("TG" %in% vars) data_daily$TG <- data_daily$TG / 10
  if ("TN" %in% vars) data_daily$TN <- data_daily$TN / 10
  if ("TX" %in% vars) data_daily$TX <- data_daily$TX / 10
  if ("RH" %in% vars) data_daily$RH <- data_daily$RH / 10
  
  
  return(data_daily)
}

##Part2: For KNMI spinup data ##

#Parameter validation - check whether all parameters were filled in by user
if (param_stationID_spinup == "" ||
    param_from_date_spinup == "" ||
    param_to_date_spinup == "" ||
    length(param_vars_spinup) == 0) {
  stop("All parameters must be provided by the user.")
}

#Split climate variables to make columns and rename so that function can use set parameter values 
param_vars_spinup <- unlist(strsplit(param_vars_spinup, ",\\s*"))
stationID  <- param_stationID_spinup
from       <- param_from_date_spinup
to         <- param_to_date_spinup

param_vars_spinup[param_vars_spinup == "average_windspeed"]     <- "FG"
param_vars_spinup[param_vars_spinup == "average_temperature"]   <- "TG"
param_vars_spinup[param_vars_spinup == "minimum_temperature"]   <- "TN"
param_vars_spinup[param_vars_spinup == "maximum_temperature"]   <- "TX"
param_vars_spinup[param_vars_spinup == "total_precipitation"]   <- "RH"
param_vars_spinup[param_vars_spinup == "average_winddirection"] <- "DDVEC"
param_vars_spinup[param_vars_spinup == "relative_humidity"]      <- "UG"
param_vars_spinup[param_vars_spinup == "minimum_humidity"]      <- "UN"
param_vars_spinup[param_vars_spinup == "maximum_humidity"]      <- "UX"

#### Part 3: Run the functions twice for daily and spinup 
KNMI_daily <- get_data(
    stationID = param_stationID,
    from      = param_from_date,
    to        = param_to_date,
    vars      = param_vars
)

KNMI_spinup <- get_data(
  stationID = param_stationID_spinup,
  from      = param_from_date_spinup,
  to        = param_to_date_spinup,
  vars      = param_vars_spinup
)


KNMI_daily_datafile <- "/tmp/data/KNMI_daily.csv"
write.csv(KNMI_daily, 
          file = KNMI_daily_datafile, 
          row.names = FALSE)

KNMI_spinup_datafile <- "/tmp/data/KNMI_spinup.csv"
write.csv(KNMI_spinup, 
          file = KNMI_spinup_datafile, 
          row.names = FALSE)

In [8]:
# KNMI data preprocessor
# ---
# NaaVRE:
#  cell:
#   inputs:
#    - KNMI_daily_datafile: String
#    - KNMI_spinup_datafile: String
#   outputs:
#    - Preprocessed_KNMI_datafile: String
#    - Preprocessed_KNMI_spinup_datafile: String
#   dependencies:
#    - name: dplyr
#    - name: stringr
#    - name: lubridate
# ...

#Read the file 
KNMI_daily <- readr::read_csv(KNMI_daily_datafile)
KNMI_spinup <- readr::read_csv(KNMI_spinup_datafile)

#Reordering the date from YYYYMMDD to Year, Month, and Day in separate columns
Preprocessed_KNMI_daily <- KNMI_daily %>%
  dplyr::mutate(YYYYMMDD = as.character(YYYYMMDD)) %>%
  dplyr::mutate(
    date  = ymd(YYYYMMDD),
    Year  = year(date),
    Month = month(date),
    Day   = day(date)
  ) %>%
  dplyr::select(-YYYYMMDD, -date) %>%
#Renaming KNMI variables to match LANDIS-II names   
  dplyr::rename(any_of(c(
    windSpeed = "FG",
    temp      = "TG",
    mintemp   = "TN",
    maxtemp   = "TX",
    precip    = "RH",
    RH        = "UG",
    maxRH     = "UX",
    minRH     = "UN"
  ))) %>%
 #Reorganize and rename columns 
tidyr::pivot_longer(
    cols = -c(Year, Month, Day),
    names_to = "Variable",
    values_to = "eco1"
  ) %>%
  dplyr::select(Year, Month, Day, Variable, eco1)

## KNMI spinup data ##
#Reordering the date from YYYYMMDD to Year, Month, and Day 
Preprocessed_KNMI_spinup <- KNMI_spinup %>%
 dplyr::mutate(YYYYMMDD = as.character(YYYYMMDD)) %>%
  dplyr::mutate(
    date  = ymd(YYYYMMDD),
    Year  = year(date),
    Month = month(date),
    Day   = day(date)
  ) %>%
  dplyr::select(-YYYYMMDD, -date) %>%
#Renaming KNMI variables to match LANDIS-II names   
  dplyr::rename(any_of(c(
    windSpeed = "FG",
    temp      = "TG",
    mintemp   = "TN",
    maxtemp   = "TX",
    precip    = "RH",
    RH        = "UG",
    maxRH     = "UX",
    minRH     = "UN"
  ))) %>%
 #Reorganize and rename columns 
  tidyr::pivot_longer(
    cols = -c(Year, Month, Day),
    names_to = "Variable",
    values_to = "eco1"
  ) %>%
  dplyr::select(Year, Month, Day, Variable, eco1)

Preprocessed_KNMI_daily_datafile  <- "/tmp/data/Preprocessed_KNMI_daily.csv"
Preprocessed_KNMI_spinup_datafile <- "/tmp/data/Preprocessed_KNMI_spinup.csv"

write.csv(Preprocessed_KNMI_daily, file = Preprocessed_KNMI_daily_datafile, row.names = FALSE)
write.csv(Preprocessed_KNMI_spinup, file = Preprocessed_KNMI_spinup_datafile, row.names = FALSE)

Rows: 7305 Columns: 10
── Column specification ────────────────────────────────────────────────────────
Delimiter: ","
dbl (10): YYYYMMDD, FG, TG, TN, TX, RH, DDVEC, UG, UN, UX

ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specify the column types or set `show_col_types = FALSE` to quiet this message.
Rows: 4017 Columns: 10
── Column specification ────────────────────────────────────────────────────────
Delimiter: ","
dbl (10): YYYYMMDD, FG, TG, TN, TX, RH, DDVEC, UG, UN, UX

ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specify the column types or set `show_col_types = FALSE` to quiet this message.


In [ ]:
# Species trait data retriever
Species_trait_datafile <- "Species_trait_datafile"

In [ ]:
# Species trait data preprocessor
Preprocessed_species_trait_datafile <- ""
Preprocessed_species_trait_datafile <- Species_trait_datafile

In [15]:
#Initial communities data retriever 
# NaaVRE:
#  cell:
#   outputs:
#    - NBI5_boomgegevens: String
#    - NBI5_proefbomen: String
#   dependencies:
#    - name: DBI
#    - name: odbc
# ...
# Load packages
install.packages("DBI")
install.packages("odbc")
library(DBI)
library(odbc)

#Step 1: Defining the path to connect with Access data base 
url_nbi <- "https://www.probos.nl/images/pdf/overig/NBI-2012-MFV-2006-v1.1.zip"
zip_path <- "/home/jovyan/Cloud Storage/naa-vre-user-data/NBI.zip"
download.file(url_nbi, zip_path, mode = "wb")
unzip(zip_path, exdir = "/home/jovyan/Cloud Storage/naa-vre-user-data/")
#-----------

#Step 2: Connect to Access database 
nbiConnect <- dbConnect(
  odbc::odbc(),
  .connection_string = paste0(
    "Driver={Microsoft Access Driver (*.mdb, *.accdb)};",
    "DBQ=/home/jovyan/Cloud Storage/naa-vre-user-data/NBI-2012-MFV-2006-v1.1.mdb;"
  )
)

NBI5_boomgegevens <- dbReadTable(nbiConnect, "data_MFV_boomgegevens")
NBI5_proefbomen   <- dbReadTable(nbiConnect, "data_MFV_Proefbomen")

Updating HTML index of packages in '.Library'

Making 'packages.html' ...
 done

Warning message in install.packages("odbc"):
“installation of package ‘odbc’ had non-zero exit status”
Updating HTML index of packages in '.Library'

Making 'packages.html' ...
 done



ERROR: Error in library(odbc): there is no package called ‘odbc’


In [14]:
## Initial community data proprocessor 
#Load libraries
library(dplyr)

#Skim unnecessary tables
Preprocessed_initial_communities <- NBI5_boomgegevens[, c(1, 4, 7)]

#Create diameter in cm column 
Preprocessed_initial_communities$diameter_cm <- Preprocessed_initial_communities$DIAMETER / 10

#Rename
names(Preprocessed_initial_communities)[names(Preprocessed_initial_communities) == 'DIAMETER'] <- 'diameter_mm'
names(Preprocessed_initial_communities)[names(Preprocessed_initial_communities) == 'PLOTNUMMER'] <- 'plotnumber'
names(Preprocessed_initial_communities)[names(Preprocessed_initial_communities) == 'BOOMSOORT'] <- 'speciesname'

names(NBI5_proefbomen)[names(NBI5_proefbomen) == 'HOOGTE'] <- 'height_m'
names(NBI5_proefbomen)[names(NBI5_proefbomen) == 'PLOTNUMMER'] <- 'plotnumber'
names(NBI5_proefbomen)[names(NBI5_proefbomen) == 'BOOMSOORT'] <- 'speciesname'

#Create table with growth factors
growthfactors <- data.frame(speciesname = c("Abeel", "Abies alba", "Abies grandis", "Amerikaanse eik", "Berk",
                                            "Beuk", "Corsicaanse den", "Corsikaanse den", "Douglas", "Es", 
                                            "Europese lariks", "Fijnspar", "Gewone esdoorn en spaanse aak", "Grove den",
                                            "Haagbeuk", "Iep", "Inlandse eik", "Italiaanse populier", "Japanse lariks",
                                            "Kastanje, Linde en Treurwilg", "Kastanje, Linde, Treurwilg", "Lijsterbes", 
                                            "Linde, kastanje en boswilg", "Omorikaspar", "Oostenrijkse den", "Pinus contorta", 
                                            "Populier", "Robinia", "Sitkaspar", "Tamme kastanje", "Thuja", "Trilpopulier", 
                                            "Tsuga","Weymouth den", "Wilg", "Zeeden", "Zoete kers", "Zwarte els"),
                                 growthfactor = c("0.591", "0.984", "0.984", "1.180", "0.787", "0.984", "0.787", "0.787", "0.984",
                                                  "0.984", "0.787", "0.984", "0.787", "0.787", "0.984", "0.591", "1.180", "0.591",
                                                  "0.787", "0.867", "0.867", "1.050", "0.867", "0.984", "0.787", "0.787", "0.591",
                                                  "0.750", "0.984", "1.180", "1.050", "0.591", "1.050", "0.787", "0.591", "0.787",
                                                  "0.787", "0.591")) 
Preprocessed_initial_communities <- Preprocessed_initial_communities %>%
  left_join(growthfactors, by = "speciesname")

#Add column with age by multiplying diameter and growth factor 
Preprocessed_initial_communities <- Preprocessed_initial_communities %>%
  mutate(
    growthfactor = as.numeric(growthfactor),
    age = diameter_cm * growthfactor
  )

#Merge the height from Proefbomen into Boomgegevens
Preprocessed_initial_communities <- Preprocessed_initial_communities %>%
  left_join(
    NBI5_proefbomen %>% select(plotnumber, speciesname, height_m),
    by = c("plotnumber", "speciesname")
  )

#There are N/As because not every species was measured in each plot 
# Calculate height/diameter_cm ratio per species from measured trees
#To deal with this, calculate a height/diameter ratio for each species 

NBI5_proefbomen <- NBI5_proefbomen %>%
  left_join(Preprocessed_initial_communities %>% select(plotnumber, speciesname, diameter_cm),
            by = c("plotnumber", "speciesname"))

# Calculate species-specific height-per-cm ratio
height_ratio_per_species <- NBI5_proefbomen %>%
  filter(!is.na(height_m) & !is.na(diameter_cm)) %>%
  mutate(height_per_cm = height_m / diameter_cm) %>%
  group_by(speciesname) %>%
  summarize(avg_height_per_cm = mean(height_per_cm, na.rm = TRUE)) %>%
  ungroup()

Preprocessed_initial_communities <- Preprocessed_initial_communities %>%
  left_join(height_ratio_per_species, by = "speciesname")

#The trees that already had height from proefbomen keep that height, the other ones 
# get height by calculating height_m = diameter_cm * avg height per cm for that species
Preprocessed_initial_communities <- Preprocessed_initial_communities %>%
  mutate(
    height_m = ifelse(is.na(height_m), diameter_cm * avg_height_per_cm, height_m)
  ) %>%
  select(plotnumber, speciesname, diameter_cm, growthfactor, age, height_m)  # keep only desired columns

#Make cohorts
cohort_size <- 10  # years per cohort

# Assign cohort number to each tree
Preprocessed_initial_communities <- Preprocessed_initial_communities %>%
  mutate(
    cohortage = ceiling(age / 10) * 10
  )

#collapse so that each plotnumber-speciesname-cohortage combination only appears once
Preprocessed_initial_community <- Preprocessed_initial_communities %>%
  group_by(plotnumber, speciesname, cohortage) %>%
  summarise(
    n_trees    = n(),
    mean_diam  = mean(diameter_cm, na.rm = TRUE),
    mean_height = mean(height_m, na.rm = TRUE),
    mean_age   = mean(age, na.rm = TRUE),
    .groups = "drop"
  )

#Rename the species to LANDIS names 
names(Preprocessed_initial_communities)[names(Preprocessed_initial_communities) == 'Abeel'] <- 'populusalba',
names(Preprocessed_initial_communities)[names(Preprocessed_initial_communities) == 'Abies alba'] <- 'abiesalba',
names(Preprocessed_initial_communities)[names(Preprocessed_initial_communities) == 'Abies grandis'] <- 'abiesgrandis',
names(Preprocessed_initial_communities)[names(Preprocessed_initial_communities) == 'Gewone esdoorn en spaanse aak'] <- 'acerpseudoplatanus',
names(Preprocessed_initial_communities)[names(Preprocessed_initial_communities) == 'Zwarte els'] <- 'alnusglutinosa',
names(Preprocessed_initial_communities)[names(Preprocessed_initial_communities) == 'Berk'] <- 'betula',
names(Preprocessed_initial_communities)[names(Preprocessed_initial_communities) == 'Haagbeuk'] <- 'carpinusbetulus', names(Preprocessed_initial_communities)[names(Preprocessed_initial_communities) == 'Tamme kastanje'] <- 'castaneasativa',
names(Preprocessed_initial_communities)[names(Preprocessed_initial_communities) == 'Chamaecyparis'] <- 'chamaecyparis',
names(Preprocessed_initial_communities)[names(Preprocessed_initial_communities) == 'Beuk'] <- 'fagussylvatica',
names(Preprocessed_initial_communities)[names(Preprocessed_initial_communities) == 'Es'] <- 'fraxinusexcelsior',
names(Preprocessed_initial_communities)[names(Preprocessed_initial_communities) == 'Europese lariks'] <- 'larixdecidua',
    names(Preprocessed_initial_communities)[names(Preprocessed_initial_communities) == 'Japanse lariks'] <- 'larixkaempfera',
    names(Preprocessed_initial_communities)[names(Preprocessed_initial_communities) == 'Fijnspar'] <- 'piceaabies',
    names(Preprocessed_initial_communities)[names(Preprocessed_initial_communities) == 'Omorikaspar'] <- 'piceaomorika',
    names(Preprocessed_initial_communities)[names(Preprocessed_initial_communities) == 'Sitkaspar'] <- 'piceasitchensis',
    names(Preprocessed_initial_communities)[names(Preprocessed_initial_communities) == 'Pinus contorta'] <- 'pinuscontorta',
    names(Preprocessed_initial_communities)[names(Preprocessed_initial_communities) == 'Corsicaanse den'] <- 'pinusnigralaricio',
    names(Preprocessed_initial_communities)[names(Preprocessed_initial_communities) == 'Corsikaanse den'] <- 'pinusnigralaricio',
    names(Preprocessed_initial_communities)[names(Preprocessed_initial_communities) == 'Oostenrijkse den'] <- 'pinusnigranigra',
    names(Preprocessed_initial_communities)[names(Preprocessed_initial_communities) == 'Zeeden'] <- 'pinuspinaster',
    names(Preprocessed_initial_communities)[names(Preprocessed_initial_communities) == 'Weymouth den'] <- 'pinusstrobus',
    names(Preprocessed_initial_communities)[names(Preprocessed_initial_communities) == 'Grove den'] <- 'pinussylvestris',
    names(Preprocessed_initial_communities)[names(Preprocessed_initial_communities) == 'Populier'] <- 'populus',
    names(Preprocessed_initial_communities)[names(Preprocessed_initial_communities) == 'Italiaanse populier'] <- 'populusnigraitalica',
    names(Preprocessed_initial_communities)[names(Preprocessed_initial_communities) == 'Trilpopulier'] <- 'populustremula',
    names(Preprocessed_initial_communities)[names(Preprocessed_initial_communities) == 'Zoete kers'] <- 'prunusavium',
    names(Preprocessed_initial_communities)[names(Preprocessed_initial_communities) == 'Douglas'] <- 'pseudotsugamenziesii',
    names(Preprocessed_initial_communities)[names(Preprocessed_initial_communities) == 'Inlandse eik'] <- 'quercusrobur',
    names(Preprocessed_initial_communities)[names(Preprocessed_initial_communities) == 'Amerikaanse eik'] <- 'quercusrubra',
    names(Preprocessed_initial_communities)[names(Preprocessed_initial_communities) == 'Robinia'] <- 'robinia',
    names(Preprocessed_initial_communities)[names(Preprocessed_initial_communities) == 'Wilg'] <- 'salix',
    names(Preprocessed_initial_communities)[names(Preprocessed_initial_communities) == 'Lijsterbes'] <- 'sorbus',
    names(Preprocessed_initial_communities)[names(Preprocessed_initial_communities) == 'Thuja'] <- 'thuja',
    names(Preprocessed_initial_communities)[names(Preprocessed_initial_communities) == 'Tsuga'] <- 'tsuga',
    names(Preprocessed_initial_communities)[names(Preprocessed_initial_communities) == 'Iep'] <- 'ulmus',
    names(Preprocessed_initial_communities)[names(Preprocessed_initial_communities) == 'Kastanje, Linde, Treurwilg'] <- 'rest',
    names(Preprocessed_initial_communities)[names(Preprocessed_initial_communities) == 'Kastanje, Linde, en Treurwilg'] <- 'rest',
    names(Preprocessed_initial_communities)[names(Preprocessed_initial_communities) == 'Kastanje, linde, en boswilg'] <- 'rest
} 

Preprocessed_initial_communities_datafile <- "/tmp/data/Preprocessed_intial_communities_datafile.csv"
write.csv(Preprocessed_initial_communities, file = Preprocessed_initial_communities_datafile, row.names = FALSE)

ERROR: Error in parse(text = input): <text>:100:113: unexpected ','
99: { 
100:     names(Preprocessed_initial_communities)[names(Preprocessed_initial_communities) == 'Abeel'] <- 'populusalba',
                                                                                                                     ^


In [ ]:
# Scenario and ecoregions builder
print(Preprocessed_KNMI_daily_datafile)
print(Preprocessed_KNMI_spinup_datafile
print(Preprocessed_species_trait_datafile)
print(Preprocessed_initial_communities_datafile)
print(Preprocessed_KNMI_spinup_datafile)
Scenario_file <- "scenario file"

In [ ]:
# LANDIS II runner
print(Scenario_file)